This is an implementation of NIPS 2017 paper, titled [Thy Friend is My Friend: Iterative Collaborative Filtering for Sparse Matrix Estimation](https://papers.nips.cc/paper/7057-thy-friend-is-my-friend-iterative-collaborative-filtering-for-sparse-matrix-estimation.pdf) for [Global NIPS Paper implementation Challenge](https://www.nurture.ai/nips-challenge). The authors have also created a short [video](https://www.youtube.com/watch?v=qxfDK44YuQE) for it.<br>
For any questions related to this notebook, feel free to contact me at ssahoo.infinity@gmail.com .

## How to use this Notebook:
Following are some important points/guidelines/assumptions to keep in mind while navigating through this notebook:
- This notebook consists of sections numbered by Roman numerals
- Brief description about all sections:
  - I: Model preparation: Changes to be made to rating matrix(dataset) before we can apply the algorithms
  - II: Algorithm Details: All the algorithms described in the paper are implemented here.
  - III: Other important functions:
    - Data Handling: for data manipulation
    - Substitutes: methods which can be used in place of (algorithm) methods in paper
    - Evaluation: for evaulation of recommender system
  - IV: Test script/Experiments: testing using a dataset
- If a function/variable has suffix as "_csr", it refers to CSR (Compressed Sparse Row) data. Else, if it has a suffix as "_matrix", it refers to 2D (numpy) matrix data.
- The dataset ratings are assumed to be integers; to be modified in future
- data_csr ensures that user_id and item_id start from 0 by taking in FIRST_INDEX as a global variable

Importing required modules required for complete notebook

In [1]:
# Built and tested on python2
import numpy as np
from tqdm import *
import sys

# 0: Model Notations
Following are the symbols/notations used in the paper. The variables/notations used in this notebook have been discussed in Experiments section.

$u$ = user <br>
$i$ = item <br>
$M$ = symmetric rating matrix of size $n \times n$ (usually the dataset) <br>
$E$ = set of $(u,i)$ where each user $u$ has rated an item $i$ also seen in the matrix $M$ (intuitively $E$ is edge set(matrix) between user and items. <br>
$p$ = sparsity of $M$ i.e. (= #observed ratings in $M$ / total # ratings in $M$)<br>
$r$ = radius, distance (in no of edges) between user $u$ and item $i$ at neighborhood boundary (look in step 2) <br>

# I: Model preparation
We first look at function which converts our asymmetric rating matrix to a symmetric matrix and another function that normalizes the ratings between [0,1].

In [2]:
'''Function to normalize all ratings of a CSR (compressed sparse row) matrix'''
def normalize_ratings_csr(data_csr):
    
    return data_csr

In [3]:
'''Function to convert asymmetrix matrix to symmetrix matrix'''
def convert_to_symmetric_matrix(data_matrix):
    #check if matrix is already symmetric?
    
    return data_matrix

# II: Algorithm Details
As per paper: *We present and discuss details of each step of the algorithm, which primarily involves computing pairwise distances (or similarities) between vertices.*

### Step 1: Sample Splitting
Partition the rating matrix into three different parts. Following are the exerpts from paper:
- *Each edge in $E$ is independently placed into $E_1, E_2,$ or $E_3$, with probabilities $c_1, c_2,$ and $1 - c_1 - c_2$ respectively. Matrices $M_1, M_2$, and $M_3$ contain information from the subset of the data in $M$ associated to $E_1, E_2$, and $E_3$ respectively.*
- *$M_1$ is used to define local neighborhoods of each vertex (in step 2), $M_2$ is used to compute similarities of these neighborhoods (in step 3), and $M_3$ is used to average over datapoints for the final estimate (in step 4)*

In [4]:
def sample_splitting_csr(data_csr, c1=0.3, c2=0.3, shuffle=False):
    if (shuffle):
        np.random.shuffle(data_csr) # inplace shuffle

    m1_sz = int(c1 * data_csr.shape[0])
    m2_sz = int(c2 * data_csr.shape[0])

    m1_csr = data_csr[              : m1_sz         ,:]
    m2_csr = data_csr[        m1_sz : m1_sz + m2_sz ,:]
    m3_csr = data_csr[m1_sz + m2_sz :               ,:]

    if m1_csr.shape[0]+m2_csr.shape[0]+m3_csr.shape[0] == data_csr.shape[0]:
        print('Sample splitting: done')
    else:
        print('Sample splitting: FAILED')
        
    return [m1_csr, m2_csr, m3_csr]

### Step 2: Expanding the Neighborhood
We do the following in this step:
- radius $r$ to be tuned using cross validation. We can use its default value as $r = \frac{6\ln(1/p)}{8\ln(c_1pn)}$ as per paper.
- use matrix $M_1$ to build neighborhood based on radius $r$
- Build BFS tree rooted at each vertex to get product of the path from user to item, such that
  - each vertex (user or item) in a path from user to boundary item is unique
  - the path chosen is the shortest path (#path edges) between the user and the boundary item
  - in case of multiple paths (or trees), choose any one path (i.e. any one tree) at random
- Normalize the product of ratings by total no of final items at the boundary

$N_{u,r}$ obtained is a vector for user $u$ for $r$-hop, where each element is product of path from user to item or zero. $\tilde{N_{u,r}}$ is normalized vector.


In [5]:
'''Function get matrix from csr such that no two item_ids and user_ids are same'''
#useful for finding paths in graph
def csr_to_matrix_with_offset(data_csr): #TODO: put it in data handling
    # Convert M1 from csr to matrix format
    ## item_ids += OFFSET
    ## so that user_ids != item_ids
    ## and we can create an undirected graph (important to get an edge from item to user)
    new_data_csr = np.copy(data_csr)
    new_data_csr[:,1] = new_data_csr[:,1] + OFFSET
    new_data_matrix = csr_to_matrix(new_data_csr, symmetry=True)
    return new_data_matrix

'''Function to create a graph as adjacency list: a dictionary of sets'''
def create_dict_graph_from_csr(data_csr):
    new_data_matrix = csr_to_matrix_with_offset(data_csr)
    # Create an (unweighted) adjacency list for the graph
    ## we still have the 2D matrix for the weights
    graph = dict()
    print('Creating graph as dictionary:')
    sys.stdout.flush()
    for i in tqdm(range(len(new_data_matrix))):
        temp_set = set()
        for j in range(len(new_data_matrix[i])):
            if new_data_matrix[i,j] > 0:
                temp_set.add(j)
        graph[i] = temp_set
    return graph


Functions useful for getting products along paths:

In [6]:
import random

'''Function gives all possible path from 'start' vertex to 'goal' vertex, inclusive of both '''
# help from:
# http://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/
# radius = # edges between start and end vertex
def bfs_paths(graph, start, radius):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for next in graph[vertex] - set(path):
            depth = len(path + [next]) - 1
            if depth == radius:
                yield path + [next]
            else:
                queue.append((next, path + [next]))

'''Function which returns a dictionary for a given user
   where each item represents the key in the dictionary
   and it returns a list of lists(paths) from user to item r-hop distance apart'''
def create_item_dict(all_path):
    dict_path = dict()
    for path in all_path:
        r_hop_item = path[-1]
        dict_path.setdefault(r_hop_item,[]).append(path) 
    return dict_path

'''Function to get product for r-hop path from user to item
   Choose any path at random if #paths > 1'''
def get_product(new_data_matrix, path):
    if len(path) < 1:
        return GET_PRODUCT_FAIL_RETURN
    idx = random.randint(0, len(path)-1)    # in case of multiple paths to same item
    p = path[idx]                           # choose any one path at random

    product = 1
    for i in range(len(p)-1):
        product = product * new_data_matrix[p[i],p[i+1]]
    return product


In [7]:
import math

'''Function to generate product matrix from user to items
   (items which are at r-hop boundary from user)'''
# if radius passed is less than 1 or not passed, this function evaluates the default radius as per paper
def generate_product_matrix(data_csr, m1_csr, c1, radius=0):
    # TODO: fix it's default radius case for symmetric matrix case
    user_list = np.array(list(set(data_csr[:,0])))
    item_list = np.array(list(set(data_csr[:,1])))
    n_ratings = len(data_csr)
    n_users = len(user_list)
    n_items = len(item_list)
    sparsity = float(    n_ratings) /  (n_users * n_items)
    
    #TODO: Find a fix for this; why is it coming less than 1?
    #print((float(6) * math.log( 1 / float(sparsity))) / (8 * math.log( c1 * sparsity * min(n_users,n_items))))
    if radius < 1:
        radius = (float(6) * math.log( 1 / float(sparsity))) / (8 * math.log( c1 * sparsity * min(n_users,n_items)))

    # First create the graph
    new_data_matrix = csr_to_matrix_with_offset(m1_csr)
    graph = create_dict_graph_from_csr(m1_csr)
    
    # Get bfs-path products from the graph
    product_matrix = np.full((len(user_list), len(item_list)), UNOBSERVED, dtype=int)
    print('Generating product matrix:')
    sys.stdout.flush()
    for user in tqdm(user_list):
        all_path = list(bfs_paths(graph, user, radius))       # 1. get a list of all r-hop paths from given user
        dict_path = create_item_dict(all_path)                # 2. create dict of paths from user to individual items
        for item in dict_path:
            paths = dict_path[item]                           # 3. get the set of user-item paths
            product = get_product(new_data_matrix, paths)       # 4. get product for a unique user-item path (at random)
            product_matrix[user, (item - OFFSET)] = product   # 5. store the product in the matrix
    return product_matrix

### Step 3: Computing the distances
Distance computation between two users (using matrix $M_2$) using the following formula (only $dist_1$ implemented for now):

$$ dist(u,v) = \left(\frac{1 - c_1p}{c_2p}\right) (\tilde{N_{u,r}} - \tilde{N_{v,r}})^T M_2 (\tilde{N_{u,r+1}} - \tilde{N_{v,r+1}}) $$

In [8]:
from scipy import stats

def generate_user_sim_matrix(data_csr, m1_csr, product_matrix):
    # making all unobserved entries in product_matrix as zero
    # makes it simpler for pearson similarity calculation, probably..
    product_matrix = find_and_replace(data=product_matrix, find_value=UNOBSERVED, replace_value=0)
    user_list = np.array(list(set(data_csr[:,0])))
    item_list = np.array(list(set(data_csr[:,1])))

    # Currently using simple pearson similarity:
    user_sim_matrix = np.full((len(user_list), len(user_list)), UNOBSERVED, dtype=float)
    print('Generating user sim matrix (pearson similarity):')
    sys.stdout.flush()
    for user1 in tqdm(user_list):
        for user2 in user_list:
            if user1 >= user2:
                [sim, p_value] = stats.pearsonr(product_matrix[user1], product_matrix[user2])
                if np.isnan(sim):                       # TODO: check if this is valid to do?
                    sim = 0
                user_sim_matrix[user1,user2] = user_sim_matrix[user2,user1] = sim
                # similarity is between -1 and 1
                # therefore, these can be directly used as weights on users' rating for prediction
    return user_sim_matrix

### Step 4: Averaging datapoints to produce final estimate
Average over nearby data points based on the distance(similarity) threshold $n_n$ (using matrix $M_3$). $n_n$ to be tuned using cross validation. Mathematically (from paper):

$$ \hat{F_{u,v}} = \frac{1}{\mid E_{uv1} \mid} \sum_{(a,b) \in E_{uv1}} M_3(a,b) $$
*where $E_{uv1}$ denotes the set of undirected edges $(a, b)$ such that $(a, b) \in E_3$ and both $dist(u, a)$ and $dist(v, b)$ are less than $n_n$*

In [9]:
def generated_unweighted_averaged_prediction_matrix(data_csr, m3_csr, user_sim_matrix, bounded=True):
    predicted_matrix = np.full((len(user_list), len(item_list)), UNOBSERVED, dtype=float)
    #actually used the matrix below for this fn
    return predicted_matrix

In [10]:
def generated_weighted_averaged_prediction_matrix(data_csr, m3_csr, user_sim_matrix, bounded=True):
    m3_matrix = csr_to_matrix(m3_csr)
    user_list = np.array(list(set(data_csr[:,0])))
    item_list = np.array(list(set(data_csr[:,1])))

    predicted_matrix = np.full((len(user_list), len(item_list)), UNOBSERVED, dtype=float)
    print('Generating prediction matrix:')
    sys.stdout.flush()
    for user in tqdm(user_list):
        for item in item_list:
            if m3_matrix[user,item] == UNOBSERVED:
                # we basically do a dot product but avoid taking UNOBSERVED user similarities or item ratings
                predicted_rating = user_sim_matrix[user, m3_matrix[:,item] != UNOBSERVED      ]\
                                    .dot(m3_matrix[      m3_matrix[:,item] != UNOBSERVED, item])
                if bounded:
                    if predicted_rating > 5:
                        predicted_rating = 5
                    elif predicted_rating < 1:
                        predicted_rating = 1
                predicted_matrix[user,item] = predicted_rating
    return predicted_matrix

# III: Other important functions

### Data Handling

In [11]:
''' Function to get data in matrix format for given data in CSR format '''
def csr_to_matrix(data_csr, symmetry=False):
    
    if symmetry:                                 ### TODO: Implement this better
        users = max(data_csr[:,0]) + 1
        items = max(data_csr[:,1]) + 1
        users = items = max(users,items)
    else:
        users = USERS
        items = ITEMS
        
    data_matrix = np.full((users, items), UNOBSERVED, dtype=int)
    for line in data_csr:
        data_matrix[line[0]][line[1]] = line[2]
        if symmetry:
            data_matrix[line[1]][line[0]] = line[2]
            
    return data_matrix

''' Function to get data in CSR format for given data in matrix format '''
def matrix_to_csr(data_matrix):             # TODO: Check if it works
    data_matrix = np.empty([0,0], dtype=int)
    data_csr = np.array([ [i,j,data_matrix[i,j]]\
                          for j in range(len(temp[i]))\
                              for i in range(len(temp))\
                                  if temp[i,j] != UNOBSERVED])
    return data_csr

''' Function to read data file, given in CSR format
    Assuming 1st 3 values of a row as: user_id, item_id, rating '''
def read_data_csr(fname, delimiter, dtype=int):
    data_csr = np.loadtxt(fname=fname, delimiter=delimiter, dtype=dtype) # Reading data to array
    data_csr = data_csr[:, :3]                                           # Extracting 1st 3 columns: 0,1,2
    data_csr[:,:2] = data_csr[:,0:2] - FIRST_INDEX                       # Making sure index starts from 0
    return data_csr

'''Function to find and replace some values
   for only 1d and 2d numpy arrays'''
def find_and_replace(data, find_value, replace_value):
    if len(data.shape) == 1:
        for i in range(len(data)):
            if data[i] == find_value:
                data[i] = replace_value
    elif len(data.shape) == 2:
        for i in range(len(data)):
            for j in range(len(data[i])):
                if data[i,j] == find_value:
                    data[i,j] = replace_value
    return data

''' Function to check dataset'''
#TODO: make it set some hardcodings
def check_dataset_csr(data_csr):
    unique_users = len(np.array(list(set(data_csr[:,0]))))
    unique_items = len(np.array(list(set(data_csr[:,1]))))
    n_users = max(data_csr[:,0]) + 1
    n_items = max(data_csr[:,1]) + 1
    
    print('USERS: ' + str(n_users))
    print('ITEMS: ' + str(n_items))
    
    # checking unrated users/items TODO: this is not possible if given data is in CSR
    if n_users != unique_users:
        print('No of users with no ratings: ' + str(n_users - unique_users))
    if n_items != unique_items:
        print('No of items with no ratings: ' + str(n_items - unique_items))
    if n_users == unique_users and n_items == unique_items:
        print('All users and items have at least one rating! Good!')
    
    #checking sparsity
    #TODO: here we assume given dataset is rectangular; UNDO this assumption
    n_ratings = data_csr.shape[0]
    sparsity_asymm = float(    n_ratings) /  (n_users * n_items)     # sparsity of given rating matrix
    sparsity_symm =  float(2 * n_ratings) / ((n_users + n_items)**2) # sparsity of large symmetricized matrix
    print('Sparsity of given matrix p: ' + str(sparsity_asymm))
    print('Sparsity of large symmetricized matrix p: ' + str(sparsity_symm))
    if sparsity_asymm <= (float(1) / max(n_users, n_items)):
        print('WARNING: For given rectangular asymmetric matrix:')
        print('       : p is not polynomially larger than 1/n.')
        print('       : Using dist1 as distance computation may not gurantee that')
        print('       : expected square error converges to zero using this paper\'s algorithm.')
    else:
        print('Asymm matrix: p is polynomially larger than 1/n, all guarantees applicable')
    if sparsity_symm <= (float(1) / ((n_users + n_items)**2)):
        print('WARNING: For generated large symmetric matrix:')
        print('       : p is not polynomially larger than 1/n.')
        print('       : Using dist1 as distance computation may not gurantee that')
        print('       : expected square error converges to zero using this paper\'s algorithm.')
    else:
        print('Sym matrix : p is polynomially larger than 1/n, all guarantees applicable')

'''Function to generate training and testing data split from given CSR data'''
def generate_train_test_split_csr(data_csr, split, shuffle=True):
    # we use data_csr as it is easy to only shuffle it and accordingly create train and test set
    if shuffle:
        np.random.shuffle(data_csr) # inplace shuffle

    train_sz = int((1 - split) * data_csr.shape[0])

    train_data_csr = data_csr[: train_sz ,:]
    test_data_csr = data_csr[train_sz : ,:]

    if train_data_csr.shape[0]+test_data_csr.shape[0] == data_csr.shape[0]:
        print('Generating train test split: done')
    else:
        print('Generating train test split: FAILED')
    return [train_data_csr, test_data_csr]

''' Function to read data file, given in matrix format '''
# TODO
def read_data_matrix():
    data_matrix = np.empty([0,0], dtype=int)
    return data_matrix


### Substitute functions
Functions which can also be used instead of algorithm specific implementations for testing purposes

### Evaluation
We evaluate our recommendation algorithm using RMSE (root mean square error). <br>
According to paper, if sparsity $p$ is polynomially larger than $n^{-1}$, i.e. if $p = n^{-1 + \epsilon}$ for $\epsilon > 0$, then we can safely use $dist_1$ distance computation formula and MSE is bounded by $O((pn)^{-1/5})$.

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def generate_true_and_test_labels(test_data_csr, predicted_matrix):
    # for all the available ratings in testset
    # and for all the predicted rating for those available rating
    # put them in two separate vectors
    y_actual  = np.full((len(test_data_csr)), UNOBSERVED, dtype=float)
    y_predict = np.full((len(test_data_csr)), UNOBSERVED, dtype=float)

    print('Generating true and test label:')
    sys.stdout.flush()
    for i in tqdm(range(len(test_data_csr))):
        testpt = test_data_csr[i]
        y_actual[i]  = testpt[2]
        y_predict[i] = predicted_matrix[testpt[0], testpt[1]]
        if y_predict[i] == UNOBSERVED:       # i.e. we could not generate a rating for this test user item pair
            y_predict[i] = AVG_RATING
    return [y_actual, y_predict]


def get_mse(y_actual, y_predict):
    mse = mean_squared_error(y_actual, y_predict)
    return mse
    
def get_rmse(y_actual, y_predict):
    rmse = sqrt(mean_squared_error(y_actual, y_predict))
    return rmse
    
def get_avg_err(y_actual, y_predict):
    avg_err = sum(abs(y_actual - y_predict)) / len(y_actual)
    return avg_err

def check_mse(data_csr, y_actual, y_predict):
    #TODO: implement it properly accounting for larger symmetric matrix
    n_users = max(data_csr[:,0]) + 1
    n_items = max(data_csr[:,1]) + 1
    n_ratings = data_csr.shape[0]
    sparsity = float(    n_ratings) /  (n_users * n_items)
    mse_upper_bound = (sparsity * max(n_users, n_items)) ** (-1 / float(5))
    mse = get_mse(y_actual, y_predict)
    if mse < mse_upper_bound:
        print('As per the discussion in the paper, MSE is bounded by O((pn)**(-1/5))')
    else:
        print('ERROR: Contrary to the discusssion in the paper, MSE is NOT bounded by O((pn)**(-1/5))')
    

# IV: Test Script / Experiment
The following jupyter notebook cells make calls to above cells to run experiments on a recommendation dataset.

### Setting constants

In [13]:
'''Dataset Parameters'''
DATA_PATH = './ml-100k/u.data' # ml-100k data set has 100k ratings, 943 users and 1682 items
#DATA_TYPE =  0              # 0: CSR format, 1: 2D matrix format  # TODO: use it
DELIMITER = "\t"             # tab separated or comma separated data format
FIRST_INDEX = 1
N_RATINGS = 100000
USERS = 943
ITEMS = 1682

In [14]:
'''Hyperparameters'''
C1 = 0.2                # probability of edges in training set going to E1
C2 = 0.3                # probability of edges in training set going to E2
C3 = 1 - C1 - C2        # probability of edges in training set going to E3
RADIUS = 3              # radius of neighborhood, radius = # edges between start and end vertex, keep it -1 to use default value given in paper
THRESHOLD = 943

#checks on parameters
if C3 <= 0:
    print('ERROR: Please set the values of C1 and C2, s.t, C1+C2 < 1')

In [15]:
'''Hardcoding values'''
OFFSET = USERS + 10                     # offset so that user_id and item_id are different in graph; keep it >= #USERS
UNOBSERVED = -1
GET_PRODUCT_FAIL_RETURN = UNOBSERVED    #TODO: This hardcoding can be removed in future
TRAIN_TEST_SPLIT = 0.2                  # %age of test ratings wrt train rating ; value in between 0 and 1
AVG_RATING = 3                          # ratings for which we dont have predicted rating

### Read and prepare the dataset

In [16]:
data_csr = read_data_csr(fname=DATA_PATH, delimiter=DELIMITER)

if data_csr.shape[0] == N_RATINGS:  # gives total no of ratings read; useful for verification
    print('Reading dataset: done')
else:
    print('Reading dataset: FAILED')
    #print( '# of missing ratings: ' + str(N_RATINGS - data_csr.shape[0]))  #TODO

Reading dataset: done


In [17]:
check_dataset_csr(data_csr=data_csr)

USERS: 943
ITEMS: 1682
All users and items have at least one rating! Good!
Sparsity of given matrix p: 0.0630466936422
Sparsity of large symmetricized matrix p: 0.0290249433107
Asymm matrix: p is polynomially larger than 1/n, all guarantees applicable
Sym matrix : p is polynomially larger than 1/n, all guarantees applicable


In [18]:
#TODO : normalize the ratings and symmtericize the given matrix

In [19]:
[train_data_csr, test_data_csr] = generate_train_test_split_csr(data_csr=data_csr, split=TRAIN_TEST_SPLIT)

Generating train test split: done


### Make predictions using THE algorithm 

##### Step 1: Sample splitting

In [20]:
[m1_csr, m2_csr, m3_csr] = sample_splitting_csr(data_csr=data_csr, c1=C1, c2=C2, shuffle=False)

Sample splitting: done


##### Step 2: Expanding the Neighborhood

In [21]:
product_matrix = generate_product_matrix(data_csr, m1_csr, c1=C1, radius=RADIUS)
#TODO: check why generating product matrix is taking about a minute longer w.r.t. rawcode

Creating graph as dictionary:


100%|██████████| 2635/2635 [00:04<00:00, 572.03it/s]

Generating product matrix:



100%|██████████| 943/943 [01:40<00:00,  9.34it/s]


##### Step 3: Computing the distances

In [22]:
user_sim_matrix = generate_user_sim_matrix(data_csr, m1_csr, product_matrix)
# del product_matrix

Generating user sim matrix (pearson similarity):


100%|██████████| 943/943 [00:52<00:00, 17.83it/s] 


##### Step 4: Averaging datapoints to produce final estimate

In [23]:
predicted_matrix = generated_weighted_averaged_prediction_matrix(data_csr, m3_csr, user_sim_matrix, bounded=True)
# del user_sim_matrix

Generating prediction matrix:


100%|██████████| 943/943 [00:49<00:00, 19.00it/s]


### Evaluate the predictions

In [24]:
[y_actual, y_predict] = generate_true_and_test_labels(test_data_csr, predicted_matrix)
# del predicted_matrix

Generating true and test label:


100%|██████████| 20000/20000 [00:00<00:00, 219212.01it/s]


In [25]:
get_rmse(y_actual, y_predict)

1.2436840434772813

In [26]:
get_avg_err(y_actual, y_predict)

0.99914999999999998

In [27]:
check_mse(data_csr, y_actual, y_predict) # TODO: this might be because the matrix considered here is not symmetric?

ERROR: Contrary to the discusssion in the paper, MSE is NOT bounded by O((pn)**(-1/5))
